# Choose your own adventure!

Use this notebook to setup your OWN domain! Since this is the least protected notebook, expect to run into errors!

# SECTION 1: Generate a regional MOM6 domain


## Step 1.1: Horizontal Grid

In [ ]:
from CrocoDash.grid import Grid

# This is one way to create a grid object. You can also check out the API documentation for more ways!
grid = Grid(
  resolution = 0,
  xstart = 0,
  lenx = 0,
  ystart = 0,
  leny = 0,
  name = "<INSERT NAME HERE>",
)

## Step 1.2: Topography

In [ ]:
from CrocoDash.topo import Topo

topo = Topo(
    grid = grid,
    min_depth = 0, # Insert your minimum depth (10m is a good value for most domains)
)

In [ ]:
# This function uses the GEBCO 2024 bathymetry to set up topography -> pass in your GEBCO bathymetry to the path!

bathymetry_path='s3://crocodile-cesm/CrocoDash/data/gebco/GEBCO_2024.zarr/'

topo.interpolate_from_file(
    file_path = bathymetry_path,
    longitude_coordinate_name="lon",
    latitude_coordinate_name="lat",
    vertical_coordinate_name="elevation"
)

In [ ]:
topo.depth.plot()

In [ ]:
# Edit your topography as you see fit!
%matplotlib ipympl
from CrocoDash.topo_editor import TopoEditor
TopoEditor(topo)

## Step 1.3: Vertical Grid

In [ ]:
from CrocoDash.vgrid import VGrid

# This is one way to make a vertical grid, but you can also check out the API documentation for more ways!
vgrid  = VGrid.hyperbolic(
    nk = 0, # Insert your number of vertical levels (e.g., 75)
    depth = topo.max_depth,
    ratio=  0, # Insert your stretching factor (e.g., 20)
)

In [ ]:
print(vgrid.dz)

In [ ]:
import matplotlib.pyplot as plt
plt.close()
# Create the plot
for depth in vgrid.z:
    plt.axhline(y=depth, linestyle='-')  # Horizontal lines

plt.ylim(max(vgrid.z) + 10, min(vgrid.z) - 10)  # Invert y-axis so deeper values go down
plt.ylabel("Depth")
plt.title("Vertical Grid")
plt.show()

# SECTION 2: Create the CESM case



## Step 2.1: Specify case name and directories:


In [ ]:
from pathlib import Path

# CESM case (experiment) name
casename = "<INSERT NAME HERE!>"

# CESM source root (Update this path accordingly!!!)
cesmroot ="/home/runner/work/CrocoGallery/CrocoGallery/CESM/"

# Place where all your input files go 
inputdir = Path.home() / "croc_input" / casename
    
# CESM case directory
caseroot = Path.home() / "croc_cases" / casename

## Step 2.2: Create the Case


In [ ]:
from CrocoDash.case import Case
case = Case(
    cesmroot = cesmroot,
    caseroot = caseroot,
    inputdir = inputdir,
    ocn_grid = grid,
    ocn_vgrid = vgrid,
    ocn_topo = topo,
    project = 'NCGD0011',
    override = True, 
    machine = "derecho",
    compset = "1850_DATM%JRA_SLND_SICE_MOM6_SROF_SGLC_SWAV"  # This is the standalone MOM6 compset! Change this to add BGC or DROF if you want either!
)

# Section 3: Prepare ocean forcing data


## Step 3.1 Configure Initial Conditions and Forcings


In [ ]:
# Update this function with all kinds of forcing options based on your compset (chlorophyll, tides, bgc, etc)! You can see all the options in the API Documentation!
case.configure_forcings(
    date_range = ["<CHOOSE DATES!> START(format %Y-%m-%d 00:00:00)","END(format %Y-%m-%d 00:00:00)"],
)

##  Step 3.3: Process forcing data

In this final step, we call the `process_forcings` method of CrocoDash to cut out and interpolate the initial condition as well as all boundaries. CrocoDash also updates MOM6 runtime parameters and CESM xml variables accordingly.

In [ ]:
case.process_forcings()

# Section 4: Build and run the case

Once you have reviewed and modified the parameters as needed, you can build and execute the case using the following commands: 
```
./case.build
./case.submit
```